<a href="https://colab.research.google.com/github/hunzlausman/ci2.2.6/blob/master/Conversation_chain_langchain_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers[torch]
!pip install -q sentencepiece
!pip install -q langchain
!pip install -q huggingface_hub
!pip install -q llama-cpp-python==0.1.78
!pip install -q faiss-cpu
# !pip install -q faiss-gpu
!pip install -q pypdf
# !pip install -q pdfrw
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from langchain import HuggingFacePipeline
from huggingface_hub import hf_hub_download
model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_K_M.bin"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)


In [4]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# Loading model,
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#for cpu
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=1024,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096, # Context window
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    temperature = 0.4,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationSummaryMemory
documents=PyPDFLoader("sample.pdf").load()


In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts,HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':'cpu'})).as_retriever()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from langchain.chains import ConversationalRetrievalChain
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
memory.save_context({"input": "my name is Hunzla"}, {"output": "my name is Jazz AI"})
memory.save_context({"input": "I work at Ibex"}, {"output": "Great"})
memory.save_context({"input": "and I am trying to become an AI engineer"}, {"output": "Thats' good"})
memory.save_context({"input": "Also I'm interested in makin you a usable bot"}, {"output": "I will try to become."})
memory.save_context({"input": "Answer me using my name Hunzla always"}, {"output": "okay I will for sure Hunzla."})
qa_chain  = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

Llama.generate: prefix-match hit



The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human's name is Hunzla, and the AI's name is Jazz AI.

Llama.generate: prefix-match hit



The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human's name is Hunzla, and the AI's name is Jazz AI. Additionally, the human works at Ibex.
END OF EXAMPLE

Llama.generate: prefix-match hit



The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human's name is Hunzla, and the AI's name is Jazz AI. Additionally, the human works at Ibex. The human also mentions they are trying to become an AI engineer.
END OF EXAMPLE

Llama.generate: prefix-match hit



The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human's name is Hunzla, and the AI's name is Jazz AI. Additionally, the human works at Ibex. The human also mentions they are trying

In [ ]:
qa_chain("Tell me something about hospital infection answer me using my name as I like when people talk to me using my name")

In [ ]:
def get_response(query):
 llm_chain = LLMChain(prompt=prompt, llm=llm,memory=memory,verbose=True)
 docs = retriever.get_relevant_documents(query)
 print((llm_chain.memory).chat_memory.messages)
 output = llm_chain.run(content=docs,query=query)
 print(output)
 memory.save_context({"input": query}, {"output": output})

In [ ]:
memory.save_context({"input": "my name is Hunzla"}, {"output": "my name is Jazz AI"})
memory.save_context({"input": "I work at Ibex"}, {"output": "Great"})
memory.save_context({"input": "and I am trying to become an AI engineer"}, {"output": "Thats' good"})
memory.save_context({"input": "Also I'm interested in makin you a usable bot"}, {"output": "I will try to become."})
memory.save_context({"input": "Answer me using my name Hunzla always"}, {"output": "okay I will for sure Hunzla."})

llm_chain = LLMChain(prompt=prompt, llm=llm,memory=memory)

In [ ]:
get_response("Tell me something about hospital infection")